In [1]:
# essentials
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
import re
import string
from collections import Counter
import pickle

# text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import config

In [2]:
sample_size = 1500

In [3]:
reviews = []
positive = []

for root, dirs, files in os.walk('./data/reviews'):
    for filename in files:
        file = open(os.path.join(root, filename))
        try:
            text = file.read()
            reviews.append(text)

            if root == "./reviews\pos":
                positive.append(True)
            else:
                positive.append(False)
        except UnicodeDecodeError:
            print(f"File: {filename} wasn't loaded successfully.")

In [4]:
reviews_df = pd.DataFrame([reviews, positive])
reviews_df = reviews_df.T
reviews_df.columns = ["review", "positive"]

In [5]:
reviews_df

,review,positive
0,Story of a man who has unnatural feelings for ...,False
1,Airport '77 starts as a brand new luxury 747 p...,False
2,This film lacked something I couldn't put my f...,False
3,"Sorry everyone,,, I know this is supposed to b...",False
4,When I was little my parents took me along to ...,False
...,...,...
24994,"Seeing as the vote average was pretty low, and...",False
24995,"The plot had some wretched, unbelievable twist...",False
24996,I am amazed at how this movie(and most others ...,False
24997,A Christmas Together actually came before my t...,False


## Preprocessing

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jjasi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
stop_words = stopwords.words('english')

In [8]:
# removing all punctuations except commas
# suspended for now because comma turned out to be most occurring token
custom_punctuation = string.punctuation.replace(',', '')
custom_punctuation = custom_punctuation.replace('.', '')

In [47]:
# TODO: add removing html parts

def general_preprocess(text):
    # removing html tags
    text_no_html = re.sub('<.*>', '', text)

    # Removing numbers
    text_number = re.sub(r'\d+', '', text_no_html)

    # Lowering text
    text_lower = text_number.lower()

    # removing punctuation
    text_no_punctuations = text_lower.translate(str.maketrans('', '', string.punctuation))

    # suspended
    # # making one type of commas
    # text_unified_commas = text_cleaned.replace('.', ',')

    return text_no_punctuations

In [48]:
reviews_df

,review,positive,clean_text,tokenized
0,Story of a man who has unnatural feelings for ...,False,story of a man who has unnatural feelings for ...,"[story, of, a, man, who, has, unnatural, feeli..."
1,Airport '77 starts as a brand new luxury 747 p...,False,airport starts as a brand new luxury plane i...,"[airport, starts, as, a, brand, new, luxury, p..."
2,This film lacked something I couldn't put my f...,False,this film lacked something i couldnt put my fi...,"[this, film, lacked, something, i, couldnt, pu..."
3,"Sorry everyone,,, I know this is supposed to b...",False,sorry everyone i know this is supposed to be a...,"[sorry, everyone, i, know, this, is, supposed,..."
4,When I was little my parents took me along to ...,False,when i was little my parents took me along to ...,"[when, i, was, little, my, parents, took, me, ..."
...,...,...,...,...
24994,"Seeing as the vote average was pretty low, and...",False,seeing as the vote average was pretty low and ...,"[seeing, as, the, vote, average, was, pretty, ..."
24995,"The plot had some wretched, unbelievable twist...",False,the plot had some wretched unbelievable twists...,"[the, plot, had, some, wretched, unbelievable,..."
24996,I am amazed at how this movie(and most others ...,False,i am amazed at how this movieand most others h...,"[i, am, amazed, at, how, this, movieand, most,..."
24997,A Christmas Together actually came before my t...,False,a christmas together actually came before my t...,"[a, christmas, together, actually, came, befor..."


In [49]:
reviews_df["clean_text"] = reviews_df.review.apply(general_preprocess)

In [50]:
reviews_df["tokenized"] = reviews_df.clean_text.apply(word_tokenize)

In [52]:
all_words = []

for tokenized_reviews in reviews_df.tokenized:
    all_words.extend(tokenized_reviews)

In [56]:
Counter(all_words)

Counter({'story': 6812,
         'of': 84540,
         'a': 95943,
         'man': 2804,
         'who': 11276,
         'has': 9520,
         'unnatural': 15,
         'feelings': 217,
         'for': 26120,
         'pig': 56,
         'starts': 652,
         'out': 9906,
         'with': 24477,
         'opening': 506,
         'scene': 2530,
         'that': 39992,
         'is': 62393,
         'terrific': 264,
         'example': 750,
         'absurd': 180,
         'comedy': 1973,
         'formal': 12,
         'orchestra': 28,
         'audience': 1116,
         'turned': 566,
         'into': 4893,
         'an': 12569,
         'insane': 142,
         'violent': 284,
         'mob': 77,
         'by': 12368,
         'the': 187192,
         'crazy': 354,
         'chantings': 1,
         'its': 15128,
         'singers': 27,
         'unfortunately': 759,
         'it': 50036,
         'stays': 102,
         'whole': 1797,
         'time': 7523,
         'no': 6849,
       

In [57]:
vocabulary = [word for word, quantity in Counter(all_words).items() if quantity >= config.MIN_COUNT]

In [58]:
vocabulary_size = len(vocabulary)

In [59]:
reviews_df

,review,positive,clean_text,tokenized
0,Story of a man who has unnatural feelings for ...,False,story of a man who has unnatural feelings for ...,"[story, of, a, man, who, has, unnatural, feeli..."
1,Airport '77 starts as a brand new luxury 747 p...,False,airport starts as a brand new luxury plane i...,"[airport, starts, as, a, brand, new, luxury, p..."
2,This film lacked something I couldn't put my f...,False,this film lacked something i couldnt put my fi...,"[this, film, lacked, something, i, couldnt, pu..."
3,"Sorry everyone,,, I know this is supposed to b...",False,sorry everyone i know this is supposed to be a...,"[sorry, everyone, i, know, this, is, supposed,..."
4,When I was little my parents took me along to ...,False,when i was little my parents took me along to ...,"[when, i, was, little, my, parents, took, me, ..."
...,...,...,...,...
24994,"Seeing as the vote average was pretty low, and...",False,seeing as the vote average was pretty low and ...,"[seeing, as, the, vote, average, was, pretty, ..."
24995,"The plot had some wretched, unbelievable twist...",False,the plot had some wretched unbelievable twists...,"[the, plot, had, some, wretched, unbelievable,..."
24996,I am amazed at how this movie(and most others ...,False,i am amazed at how this movieand most others h...,"[i, am, amazed, at, how, this, movieand, most,..."
24997,A Christmas Together actually came before my t...,False,a christmas together actually came before my t...,"[a, christmas, together, actually, came, befor..."


In [61]:
reviews_df.to_csv("data/reviews_cleaned.csv", index=False)

In [62]:
small_reviews_df = reviews_df.sample(sample_size)
small_reviews_df.to_csv("data/reviews_cleaned_sample.csv", index=False)